<a href="https://colab.research.google.com/github/jmk9/Deeplearning_seminar/blob/master/ch13_JHpart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Part 1: Introduction to Tensors**



> Tensor class가 모든 수치 정보를 Numpy 배열(self.data)에 담고, 한 가지 텐서 연산을 지원





In [1]:

import numpy as np

class Tensor (object):
    
    def __init__(self, data):
        self.data = np.array(data)
    
    def __add__(self, other):
        return Tensor(self.data + other.data)
    
    def __repr__(self):
        return str(self.data.__repr__())
    
    def __str__(self):
        return str(self.data.__str__())
    
x = Tensor([1,2,3,4,5])
print(x)

[1 2 3 4 5]


In [2]:
y = x + x
print(y)

[ 2  4  6  8 10]


# **Part 2: Introduction to Autograd**

## **역전파 자동화 하기**
> 일단 신경망 출력에서 기울기를 계산한 뒤, 이 계산 결과를 끝에서 두 번째 구성요소의 미분계수 계산에 이용합니다. 그리고 아키텍처 내부의 모든 가중치가 정확한 기울기를 갖게 될 때까지 이 과정을 반복합니다.



In [0]:
import numpy as np

class Tensor (object):
    
    def __init__(self, data, creators=None, creation_op = None):
        self.data = np.array(data)
        self.creation_op = creation_op
        self.creators = creators
        self.grad = None
    
    def backward(self, grad):                ## 자동 미분
        self.grad = grad
        
        if(self.creation_op == "add"):
            self.creators[0].backward(grad)
            self.creators[1].backward(grad)

    def __add__(self, other):
        return Tensor(self.data + other.data,  creators=[self,other], creation_op="add")
    
    def __repr__(self):
        return str(self.data.__repr__())
    
    def __str__(self):
        return str(self.data.__str__())

x = Tensor([1,2,3,4,5])
y = Tensor([2,2,2,2,2])

z = x + y
z.backward(Tensor(np.array([1,1,1,1,1])))

In [15]:
print(x.grad)
print(y.grad)
print(z.creators)
print(z.creation_op)

[1 1 1 1 1]
[1 1 1 1 1]
[array([1, 2, 3, 4, 5]), array([2, 2, 2, 2, 2])]
add



> ### autorad에서는 각 벡터가 자신의 모든 `self.creators`에 대해 `.backward()`를 호출하기 때문에 재귀적으로 동작함.



### * 재귀적 : 자기 자신을 다시 이용하여 정의하거나 응용하는 것

In [9]:
a = Tensor([1,2,3,4,5])
b = Tensor([2,2,2,2,2])
c = Tensor([5,4,3,2,1])
d = Tensor([-1,-2,-3,-4,-5])

e = a + b
f = c + d
g = e + f
g.backward(Tensor(np.array([1,1,1,1,1])))

print(a.grad)

[1 1 1 1 1]


## **Part 3: Tensors That Are Used Multiple Times** 


> Tensor의 현재 버전은 변수로의 역전파를 1회만 지원합니다.
> 하지만 순전파를 하는 동안 텐서 (신경망의 가중치) 하나를 여러 번 재사용해야 하고 그래프의 각 부분은 경사도를 동일 텐서 안으로 역전파 합니다.
> 그러나 코드는 현재 여러 번 사용된 변수로 역전파 하면 경사도를 정확히 계산할 수 없습니다.



In [10]:
a = Tensor([1,2,3,4,5])
b = Tensor([2,2,2,2,2])
c = Tensor([5,4,3,2,1])

d = a + b
e = b + c   # b 재사용됨
f = d + e
f.backward(Tensor(np.array([1,1,1,1,1])))

b.grad.data == np.array([2,2,2,2,2])

array([False, False, False, False, False])

## **Part 4: Upgrading Autograd to Support Multiple Tensors**


###2가지 기능 추가!

1.**경사도를 누적해서 변수가 한 번 이상 사용될 때 해당 변수가 모든 자식으로부터 경사도를 수신할 수 있도록 하는 기능**
+) self.Children counter를 생성 --> 의도치 않게 같은 자식에서 두 번 이뤄지는 역전파로부터 변수 보호 가능

2.**all_children_grads_accounted_for() 함수 추가.** 텐서가 graph 내의 모든 children으로부터 경사도를 수신했는지 계산하는 것.



In [11]:
import numpy as np

class Tensor (object):
    
    def __init__(self,data,
                 autograd=False,
                 creators=None,
                 creation_op=None,
                 id=None):
        
        self.data = np.array(data)
        self.autograd = autograd
        self.grad = None
        if(id is None):
            self.id = np.random.randint(0,100000)
        else:
            self.id = id
        
        self.creators = creators
        self.creation_op = creation_op
        self.children = {}
        
        if(creators is not None):
            for c in creators:
                if(self.id not in c.children):          # tensor가 얼마나 많은 자식을 갖고 있는지 계속 추적
                    c.children[self.id] = 1
                else:
                    c.children[self.id] += 1

    def all_children_grads_accounted_for(self):         # tensor가 각 자식으로부터 수신한 경사도 개수가 정확한지 확인
        for id,cnt in self.children.items():
            if(cnt != 0):
                return False
        return True        
        
    def backward(self,grad=None, grad_origin=None):
        if(self.autograd):
            if(grad is None):
                grad = FloatTensor(np.ones_like(self.data))
            
            if(grad_origin is not None):
                if(self.children[grad_origin.id] == 0):     # 역전파가 가능한지 또는 경사도를 기다리고 있는 상태인지를 확인해서
                    raise Exception("cannot backprop more than once") # ㅎ경사도를 기다리고 있는 상태라면 카운터를 감소시킴
                else:
                    self.children[grad_origin.id] -= 1

            if(self.grad is None):
                self.grad = grad                            # 여러 자식의 경사도를 누적함.
            else:
                self.grad += grad
            
            # grads must not have grads of their own
            assert grad.autograd == False
            
            # only continue backpropping if there's something to
            # backprop into and if all gradients (from children)
            # are accounted for override waiting for children if
            # "backprop" was called on this variable directly
            if(self.creators is not None and 
               (self.all_children_grads_accounted_for() or 
                grad_origin is None)):

                if(self.creation_op == "add"):
                    self.creators[0].backward(self.grad, self)
                    self.creators[1].backward(self.grad, self)        # 실제 역전파를 시작
                    
    def __add__(self, other):
        if(self.autograd and other.autograd):
            return Tensor(self.data + other.data,
                          autograd=True,
                          creators=[self,other],
                          creation_op="add")
        return Tensor(self.data + other.data)

    def __repr__(self):
        return str(self.data.__repr__())
    
    def __str__(self):
        return str(self.data.__str__())  
    
a = Tensor([1,2,3,4,5], autograd=True)
b = Tensor([2,2,2,2,2], autograd=True)
c = Tensor([5,4,3,2,1], autograd=True)

d = a + b
e = b + c
f = d + e

f.backward(Tensor(np.array([1,1,1,1,1])))

print(b.grad.data == np.array([2,2,2,2,2]))

[ True  True  True  True  True]
